# 使用 LCEL 构建一个简单的 LLM 应用程序

在这个快速入门中，我们将用 LangChain 构建一个简单的 LLM 应用程序。

这个应用程序将把文本从英语翻译成另一种语言。

这是一个相对简单的 LLM 应用程序 - 它只是一次 LLM 调用加上一些提示。

阅读完本教程后，您将对以下内容有一个高层次的概述：

- 使用 **语言模型**
- 使用 **PromptTemplates** 和 **OutputParsers**
- 使用 **LangChain** 表达语言 **LCEL** 将组件链在一起



## Install

In [ ]:
# 安装主要的 LangChain 库
% pip install langchain

# LangChain 核心库
% pip install langchain-core

# 语言模型
% pip install langchain-openai

# 尚未拆分为单独包的集成 langchain-community 包
% pip install langchain-community


# 使用语言模型
## 不用 **LangChain**, 直接调用模型接口

### OpenAI

```shell
pip install openai
```

配置 OpenAI API Key

```shell
export OPENAI_API_KEY="sk-proj-..."
```

In [1]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Hello, how are you?"
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', function_call=None, tool_calls=None)


### DeepSeek

```shell
export DEEPSEEK_API_KEY="sk-..."
```


In [2]:
import os
from openai import OpenAI

api_key = os.getenv("DEEPSEEK_API_KEY")

client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "role": "user",
            "content": "Hello, how are you?"
        },
    ],
)

print(response.choices[0].message)

ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?", refusal=None, role='assistant', function_call=None, tool_calls=None)


## 使用 LangChain 调用模型
### OpenAI

```shell
pip install langchain-openai
```

```shell
export OPENAI_API_KEY="sk-proj-..."
```

- `ChatModel` 是 LangChain "Runnables" 的实例，这意味着它们暴露了一个标准接口供我们与之交互。
- 要简单地调用模型，我们可以将消息列表传递给 `.invoke` 方法。

In [3]:
from langchain_openai import ChatOpenAI

openai_model = ChatOpenAI(model="gpt-4o-mini")

response = openai_model.invoke(["Hello, how are you?"])
response

AIMessage(content="Hello! I'm just a program, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 13, 'total_tokens': 36, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a129c1d-5df0-4460-9a9d-fc1746b7f4a9-0', usage_metadata={'input_tokens': 13, 'output_tokens': 23, 'total_tokens': 36})

In [4]:
print(response)

content="Hello! I'm just a program, but I'm here and ready to help you. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 13, 'total_tokens': 36, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None} id='run-8a129c1d-5df0-4460-9a9d-fc1746b7f4a9-0' usage_metadata={'input_tokens': 13, 'output_tokens': 23, 'total_tokens': 36}


### DeepSeek

```shell
export DEEPSEEK_API_KEY="sk-..."
```

In [5]:
import os
from langchain_openai import ChatOpenAI

api_key = os.getenv("DEEPSEEK_API_KEY")

deepseek_model = ChatOpenAI(
    model="deepseek-chat",
    openai_api_key=api_key,
    openai_api_base='https://api.deepseek.com'
)

response = deepseek_model.invoke(["Hello, how are you?"])
response

AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 9, 'total_tokens': 45, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_1c141eb703', 'finish_reason': 'stop', 'logprobs': None}, id='run-577ce806-9d1b-4028-bb00-5bdf7216abea-0', usage_metadata={'input_tokens': 9, 'output_tokens': 36, 'total_tokens': 45})

In [6]:
print(response)

content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 9, 'total_tokens': 45, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_1c141eb703', 'finish_reason': 'stop', 'logprobs': None} id='run-577ce806-9d1b-4028-bb00-5bdf7216abea-0' usage_metadata={'input_tokens': 9, 'output_tokens': 36, 'total_tokens': 45}


### Messages

- 消息是 LangChain 中用于与 LLM 交互的主要数据结构。
- 它们是包含角色（如user、assistant或system）和内容（如文本）的字典列表。

    - `HumanMessage` 用户发送的消息
    - `AIMessage` LLM 发送的消息
    - `SystemMessage` 系统消息

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="Hello, how are you?"),
]

response = openai_model.invoke(messages)
response

AIMessage(content='你好，你好吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 24, 'total_tokens': 28, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-ec08491c-eba7-4048-8889-08a84faac466-0', usage_metadata={'input_tokens': 24, 'output_tokens': 4, 'total_tokens': 28})

In [8]:
response = deepseek_model.invoke(messages)
response

AIMessage(content='你好，你好吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 16, 'total_tokens': 21, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 16}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_1c141eb703', 'finish_reason': 'stop', 'logprobs': None}, id='run-681d9bfd-2f08-4d61-8ffc-cd372651a58f-0', usage_metadata={'input_tokens': 16, 'output_tokens': 5, 'total_tokens': 21})

# OutputParser 输出解析器

模型的响应是一个 `AIMessage`。这包含一个字符串响应以及有关响应的其他元数据。通常我们可能只想处理字符串响应。我们可以通过使用简单的输出解析器来解析出这个响应。
- 输出解析器是 LangChain 中用于解析 LLM 输出的组件。
- 它们将 LLM 的输出转换为更有用的格式，例如字符串、JSON、Python 对象等。
- 输出解析器在 LangChain 中广泛使用，用于处理和操作 LLM 的输出。

In [9]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parser.invoke(response)

'你好，你好吗？'

In [10]:
response.content

'你好，你好吗？'

# PromptTemplate 提示模板

现在我们直接将消息列表传递给语言模型。这些消息列表来自哪里？通常，它是由用户输入和应用程序逻辑的组合构建的。此应用程序逻辑通常将原始用户输入转换为准备传递给语言模型的消息列表。常见的转换包括添加系统消息或使用用户输入格式化模板。

PromptTemplates 是 LangChain 中的一个概念，旨在帮助进行这种转换。它们接受原始用户输入并返回数据（提示），这些数据准备好传递给语言模型。

让我们在这里创建一个 PromptTemplate。它将接受两个用户变量：

- `language`: 要翻译文本的语言
- `text`: 要翻译的文本

In [11]:
from langchain_core.prompts import ChatPromptTemplate


prompt_template = ChatPromptTemplate(
    [
        ("system", "Translate the following into {language}:"),
        ("user", "{text}")
    ]
)
result = prompt_template.invoke(
    {
        "language": "Chinese",
        "text": "Hello, how are you?"
    }
)
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Chinese:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

In [12]:
print(result)

messages=[SystemMessage(content='Translate the following into Chinese:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]


我们可以看到它返回一个 `ChatPromptValue`，由两条消息组成。
如果我们想直接访问消息，我们可以这样做：

In [13]:
print(result.messages)
result.to_messages()

[SystemMessage(content='Translate the following into Chinese:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]


[SystemMessage(content='Translate the following into Chinese:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

# 使用 LCEL 将组件链在一起

我们现在可以使用管道（`|`）运算符将其与上面的提示模板、模型和输出解析器结合起来：


In [14]:
chain = prompt_template | openai_model | parser

chain.invoke(
    {
        "language": "Chinese",
        "text": "Hello, how are you?"
    }
)

'你好，你好吗？'

In [15]:
chain = prompt_template | deepseek_model | parser

chain.invoke({"language": "Chinese", "text": "Hello, how are you?"})

'你好，你好吗？'